In [1]:
from glob import glob

import cmath
import matplotlib.pyplot as plt  
import matplotlib as mpl  
import numpy as np  
import openslide  
from openslide import open_slide  
from openslide.deepzoom import DeepZoomGenerator  
import pandas as pd

from skimage.color import rgb2gray 
from skimage import color, morphology, filters, exposure, feature
import cv2
from scipy import ndimage
from PIL import Image

In [ ]:
# Get tiles from tissue region only

# Use OTSU threshold value from entire image ( level = 8) 
# use this threshold value to get foreground image at each tile at level_count -2

files = glob("Data/Stage4/*.svs")  
numSlide = 1;
for file in files:
    slide = open_slide(file)
    tile_size = 256
    tiles = DeepZoomGenerator(slide, tile_size=tile_size, overlap=0, limit_bounds=False)  
    
    # Get mask for entire slide. Identify tissue region
    level = 8
    
    tile = tiles.get_tile(level, (0,0))
            
    threshold_value = filters.threshold_otsu(np.array(tile))
            
    level = tiles.level_count - 2
    for i in range(tiles.level_tiles[level][0]):
        for j in range(tiles.level_tiles[level][1]):
            tile = tiles.get_tile(level, (i,j))
            tile_otsu = tile >= threshold_value
            tile_filter = ndimage.filters.median_filter(tile_otsu, size = 3)

            kernel = np.ones((10,10))
            tile_filter = cv2.dilate(tile_filter.astype(np.float32),kernel,iterations =1)
            tile_filter = cv2.erode(tile_filter.astype(np.float32),kernel,iterations =1)

            lower = np.array([0,0,0])
            upper = np.array([200,0,200])
            mask = cv2.inRange(tile_filter, lower, upper)
            mask = np.repeat(mask[:, :, np.newaxis], 3, axis=2)
            
            ratio = float(np.sum(mask==255)) / (256*256*3)
            
            if ratio > 0.5:
                tile.save('tiles/Stage4/tileStage4Slide' + str(numSlide) + "-" + str(i) + "-" + str(j) + ".png" , "png")
        
            
   
    numSlide = numSlide + 1
   
    
    